# Reproduction of Adult dataset experiments

In this notebook we reproduce the results from Table 2 of the DECAF paper. We compare various methods for generating debiased data using the DECAF model against synthetic data generated using benchmark models GAN, WGAN-GP and FairGAN. As described in the paper we run all experiments (as implemented in this notebook) 10 times and avarage the results.

In [1]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

from data import load_adult, preprocess_adult
from metrics import DP, FTU
from train import train_decaf, train_fairgan, train_vanilla_gan, train_wgan_gp


## Loading data

In [2]:
dataset = load_adult()
dataset.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


Preprocess the data next in order to make it suitable for training models on.

In [3]:
dataset = preprocess_adult(dataset)
dataset.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.301370,0.833333,0.043338,0.000000,0.800000,0.333333,0.615385,0.6,0.0,1.0,0.02174,0.0,0.397959,0.0,1.0
1,0.452055,0.166667,0.047277,0.000000,0.800000,0.000000,0.307692,0.4,0.0,1.0,0.00000,0.0,0.122449,0.0,1.0
2,0.287671,0.000000,0.137244,0.200000,0.533333,0.166667,0.461538,0.6,0.0,1.0,0.00000,0.0,0.397959,0.0,1.0
3,0.493151,0.000000,0.150212,0.133333,0.400000,0.000000,0.461538,0.4,1.0,1.0,0.00000,0.0,0.397959,0.0,1.0
4,0.150685,0.000000,0.220703,0.000000,0.800000,0.000000,0.384615,0.0,1.0,0.0,0.00000,0.0,0.397959,0.3,1.0


Split the dataset into train and test folds. Test fold size is 2000.

In [4]:
# Split data into train and testing sets
dataset_train, dataset_test = train_test_split(dataset, test_size=2000,
                                               stratify=dataset['income'])

print('Size of train set:', len(dataset_train))
print('Size of test set:', len(dataset_test))

Size of train set: 28162
Size of test set: 2000


### Defining the DAG

We need to define a DAG which captures the biases of the dataset. As described in the DECAF paper normally a causal discovery algorithm is used. In this notebook we simply copy the DAG which as described in the Zhang et al. paper which is the one also used in the DECAF paper.

In [5]:
# Define DAG for Adult dataset
dag = [
    # Edges from race
    ['race', 'occupation'],
    ['race', 'income'],
    ['race', 'hours-per-week'],
    ['race', 'education'],
    ['race', 'marital-status'],

    # Edges from age
    ['age', 'occupation'],
    ['age', 'hours-per-week'],
    ['age', 'income'],
    ['age', 'workclass'],
    ['age', 'marital-status'],
    ['age', 'education'],
    ['age', 'relationship'],
    
    # Edges from sex
    ['sex', 'occupation'],
    ['sex', 'marital-status'],
    ['sex', 'income'],
    ['sex', 'workclass'],
    ['sex', 'education'],
    ['sex', 'relationship'],
    
    # Edges from native country
    ['native-country', 'marital-status'],
    ['native-country', 'hours-per-week'],
    ['native-country', 'education'],
    ['native-country', 'workclass'],
    ['native-country', 'income'],
    ['native-country', 'relationship'],
    
    # Edges from marital status
    ['marital-status', 'occupation'],
    ['marital-status', 'hours-per-week'],
    ['marital-status', 'income'],
    ['marital-status', 'workclass'],
    ['marital-status', 'relationship'],
    ['marital-status', 'education'],
    
    # Edges from education
    ['education', 'occupation'],
    ['education', 'hours-per-week'],
    ['education', 'income'],
    ['education', 'workclass'],
    ['education', 'relationship'],
    
    # All remaining edges
    ['occupation', 'income'],
    ['hours-per-week', 'income'],
    ['workclass', 'income'],
    ['relationship', 'income'],
]

def dag_to_idx(df, dag):
    """Convert columns in a DAG to the corresponding indices."""

    dag_idx = []
    for edge in dag:
        dag_idx.append([df.columns.get_loc(edge[0]), df.columns.get_loc(edge[1])])

    return dag_idx

# Convert the DAG to one that can be provided to the DECAF model
dag_seed = dag_to_idx(dataset, dag)
print(dag_seed)

[[8, 6], [8, 14], [8, 12], [8, 3], [8, 5], [0, 6], [0, 12], [0, 14], [0, 1], [0, 5], [0, 3], [0, 7], [9, 6], [9, 5], [9, 14], [9, 1], [9, 3], [9, 7], [13, 5], [13, 12], [13, 3], [13, 1], [13, 14], [13, 7], [5, 6], [5, 12], [5, 14], [5, 1], [5, 7], [5, 3], [3, 6], [3, 12], [3, 14], [3, 1], [3, 7], [6, 14], [12, 14], [1, 14], [7, 14]]


It's also necessary to define edges we want to remove from the DAG in order to meet the various fairness criteria described in the paper.

In [6]:
def create_bias_dict(df, edge_map):
    """
    Convert the given edge tuples to a bias dict used for generating
    debiased synthetic data.
    """
    bias_dict = {}
    for key, val in edge_map.items():
        bias_dict[df.columns.get_loc(key)] = [df.columns.get_loc(f) for f in val]
    
    return bias_dict

# Bias dictionary to satisfy FTU
bias_dict_ftu = create_bias_dict(dataset, {'income': ['sex']})
print('Bias dict FTU:', bias_dict_ftu)

# Bias dictionary to satisfy DP
bias_dict_dp = create_bias_dict(dataset, {'income': [
    'occupation', 'hours-per-week', 'marital-status', 'education', 'sex',
    'workclass', 'relationship']})
print('Bias dict DP:', bias_dict_dp)

# Bias dictionary to satisfy CF
bias_dict_cf = create_bias_dict(dataset, {'income': [
    'marital-status', 'sex']})
print('Bias dict CF:', bias_dict_cf)

Bias dict FTU: {14: [9]}
Bias dict DP: {14: [6, 12, 5, 3, 9, 1, 7]}
Bias dict CF: {14: [5, 9]}


## Experiments

We have loaded and preprocessed the data and we are ready to run the experiments. For each experiment we train a generative model, sample synthetic data from the trained model and then obtain metrics by training and evaluating a downstream multi-layer perceptron using the test fold we generated in the previous section. We use the MLP model from `sklearn` with default parameters which matches the settings described in Appendix D of the paper.

In [7]:
def eval_model(dataset_train, dataset_test):
    """Helper function that prints evaluation metrics."""

    X_train, y_train = dataset_train.drop(columns=['income']), dataset_train['income']
    X_test, y_test = dataset_test.drop(columns=['income']), dataset_test['income']

    clf = MLPClassifier()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    auroc = roc_auc_score(y_test, y_pred)
    dp = DP(clf, X_test)
    ftu = FTU(clf, X_test)

    return {'precision': precision, 'recall': recall, 'auroc': auroc,
            'dp': dp, 'ftu': ftu}

### Original dataset

As a benchmark we want to first train the downstream model on the original dataset.

In [8]:
eval_model(dataset_train, dataset_test)

{'precision': 0.8855305466237942,
 'recall': 0.9167776298268975,
 'auroc': 0.7796739554756977,
 'dp': 0.20678740907675108,
 'ftu': 0.02200000000000002}

In the following sections we train various models in order to reproduce the results from Table 2 of the DECAF paper.

### GAN

In [9]:
synth_data = train_vanilla_gan(dataset_train)
synth_data.head()

2022-02-02 13:45:58.928530: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-02-02 13:45:58.956530: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Synthetic data generation: 100%|██████████| 221/221 [00:02<00:00, 102.89it/s]


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,-0.005068,0.166667,0.001333,0.133333,0.066667,0.166667,0.615385,0.4,0.25,1.0,-0.002291,0.006366,0.005182,0.350,0.0
1,-0.001015,0.500000,0.005365,0.000000,0.866667,0.000000,1.000000,0.2,0.50,1.0,-0.004930,0.007240,0.008546,0.275,1.0
2,-0.005068,0.000000,0.001333,0.733333,0.266667,0.500000,0.307692,0.6,0.00,0.0,-0.002291,0.006366,0.005182,0.425,1.0
3,0.002939,0.000000,0.007170,0.866667,0.133333,0.166667,0.692308,0.4,1.00,1.0,-0.005566,0.007833,0.011496,0.250,0.0
4,-0.005068,0.166667,0.001333,0.133333,0.666667,0.500000,0.230769,0.6,0.25,0.0,-0.002291,0.006366,0.005182,0.750,0.0


In [10]:
eval_model(synth_data, dataset_test)

{'precision': 0.7304925290536801,
 'recall': 0.8788282290279628,
 'auroc': 0.4504582912208087,
 'dp': 0.11255702742813167,
 'ftu': 0.12549999999999994}

### WGAN-GP

In [11]:
synth_data = train_wgan_gp(dataset_train)
synth_data.head()

Synthetic data generation: 100%|██████████| 57/57 [00:00<00:00, 72.45it/s]


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.432468,0.333333,0.088424,0.666667,0.200000,0.833333,0.307692,0.0,1.00,0.0,0.045116,0.048623,0.354788,0.800,0.0
1,0.339247,0.833333,0.075099,0.600000,0.400000,0.833333,0.769231,1.0,0.75,0.0,0.022114,0.039937,0.329692,0.625,1.0
2,0.280273,0.833333,0.061916,0.333333,0.533333,0.000000,1.000000,0.0,0.75,1.0,0.014650,0.040334,0.324537,0.400,1.0
3,0.740817,0.666667,0.171872,1.000000,0.333333,1.000000,0.846154,1.0,1.00,1.0,0.124001,0.062172,0.524774,0.875,1.0
4,0.395729,0.500000,0.094245,0.066667,0.666667,0.000000,0.615385,1.0,0.00,1.0,0.026202,0.046144,0.363207,1.000,1.0


In [12]:
eval_model(synth_data, dataset_test)

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


{'precision': 0.7737068965517241,
 'recall': 0.7170439414114514,
 'auroc': 0.5422569104647619,
 'dp': 0.062349988090657615,
 'ftu': 0.018999999999999906}

### FairGAN

In [13]:
synth_data = train_fairgan(dataset_train)
synth_data.head()

INFO:tensorflow:Restoring parameters from cache/fairgan
burning in


2022-02-02 13:46:42.126856: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)


generating


/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.000000,0.369636,0.429304,0.568245,0.522931,0.404446,0.595719,0.000000,0.528495,0.0,0.000000,0.691437,0.000000,0.582464,1.0
1,0.513192,0.168188,0.500070,0.927547,0.000000,0.549738,0.565060,0.000000,0.711314,0.0,0.219476,0.433434,0.000000,0.000000,1.0
2,1.287684,0.025203,0.422490,1.111596,0.000000,0.458210,0.021143,0.000000,1.612983,0.0,0.000000,0.689238,0.000000,0.000000,1.0
3,0.000000,0.041467,0.410660,0.737548,0.000000,0.000000,0.614550,0.000000,0.712770,0.0,0.000000,0.520030,0.000000,0.229288,1.0
4,0.000000,0.175688,0.000000,0.529809,0.000000,0.218120,0.360827,0.300408,0.614899,0.0,0.277231,0.262351,0.107554,0.506268,0.0


In [14]:
eval_model(synth_data, dataset_test)

{'precision': 0.8881401617250674,
 'recall': 0.877496671105193,
 'auroc': 0.7720816688859299,
 'dp': 0.32327451950383845,
 'ftu': 0.22399999999999998}

### DECAF

#### DECAF-ND

In [15]:
synth_data = train_decaf(dataset_train, dag_seed)
synth_data.head()

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
9389,0.548157,7.198008e-04,0.040858,0.071366,0.596733,4.051269e-06,0.781310,0.383696,1.067760e-31,1.0,4.090571e-04,1.306405e-10,0.701891,4.142208e-02,0.0
12010,0.444242,7.133225e-01,0.099510,0.277705,0.554052,3.254546e-07,0.143423,0.388937,5.769578e-04,1.0,2.807623e-05,2.223166e-09,0.630247,2.729287e-21,1.0
16562,0.586892,1.548638e-04,0.148240,0.199428,0.587697,7.452913e-06,0.170560,0.077885,9.321551e-10,0.0,1.198374e-11,6.939888e-09,0.405021,1.815254e-04,1.0
25464,0.080151,1.258729e-03,0.043055,0.128660,0.610267,3.445886e-01,0.548782,0.866987,2.136185e-31,0.0,8.220196e-15,6.115080e-09,0.333527,1.508157e-16,1.0
23379,0.422758,7.147251e-07,0.242473,0.055861,0.476768,7.987779e-07,0.314865,0.346537,1.533620e-16,1.0,1.031829e-17,1.269081e-11,0.383738,7.275611e-11,0.0


In [16]:
eval_model(synth_data, dataset_test)

{'precision': 0.9011164274322169,
 'recall': 0.7523302263648469,
 'auroc': 0.751667121214552,
 'dp': 0.37379766,
 'ftu': 0.08850002}

#### DECAF-FTU

In [17]:
synth_data = train_decaf(dataset_train, dag_seed, biased_edges=bias_dict_ftu)
synth_data.head()

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
9389,0.229924,0.058304,0.071983,0.075811,0.591183,3.449376e-01,0.698967,0.602450,4.214470e-12,1.0,4.432179e-12,1.182659e-17,0.328272,9.712963e-13,1.0
12010,0.404818,0.001039,0.035611,0.128869,0.439225,6.520313e-10,0.409505,0.395732,4.250205e-17,1.0,6.723822e-05,2.125345e-07,0.304583,3.448978e-17,0.0
16562,0.587542,0.000056,0.072627,0.923161,0.611813,3.185534e-09,0.117951,0.303437,2.208833e-17,1.0,6.048147e-10,2.808487e-12,0.464521,5.902267e-01,1.0
25464,0.273349,0.000200,0.144352,0.259049,0.872800,3.780260e-01,0.306474,0.571708,6.744551e-07,0.0,3.596466e-14,2.189424e-14,0.326169,9.281682e-22,1.0
23379,0.085595,0.000004,0.178737,0.049220,0.605434,3.403370e-01,0.100312,0.124287,2.681055e-14,1.0,6.537674e-17,4.126725e-07,0.453764,8.535411e-09,0.0


In [18]:
eval_model(synth_data, dataset_test)

{'precision': 0.8755656108597285,
 'recall': 0.7729693741677763,
 'auroc': 0.7208220364814786,
 'dp': 0.28634638,
 'ftu': 0.013999999}

#### DECAF-CF

In [19]:
synth_data = train_decaf(dataset_train, dag_seed, biased_edges=bias_dict_cf)
synth_data.head()

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
9389,0.564691,5.381976e-03,0.086409,0.212411,0.616144,7.769055e-10,0.757119,0.511164,9.988796e-01,1.0,5.314019e-09,2.369922e-16,0.271461,1.752018e-04,1.0
12010,0.070751,1.733690e-09,0.167409,0.118932,0.512686,2.957470e-01,0.442540,0.242772,2.080208e-11,1.0,1.550994e-12,8.064918e-13,0.240342,1.711320e-06,1.0
16562,0.057722,1.112474e-05,0.085405,0.300902,0.530621,3.700543e-01,0.043483,0.941804,5.630588e-07,0.0,2.706730e-14,3.524480e-12,0.338291,5.539295e-17,1.0
25464,0.395401,6.597233e-01,0.216052,0.223222,0.612192,3.347380e-01,0.461426,0.174316,6.330883e-23,0.0,4.441878e-05,1.308708e-02,0.271210,8.372100e-03,1.0
23379,0.098038,2.232730e-02,0.035785,0.114714,0.526242,2.867374e-01,0.296281,0.515785,4.313925e-07,0.0,4.060616e-07,1.334051e-03,0.227572,3.575857e-10,1.0


In [20]:
eval_model(synth_data, dataset_test)

{'precision': 0.8154235145385588,
 'recall': 0.8588548601864181,
 'auroc': 0.6362547393301569,
 'dp': 0.14107072,
 'ftu': 0.03550005}

#### DECAF-DP

In [21]:
synth_data = train_decaf(dataset_train, dag_seed, biased_edges=bias_dict_dp)
synth_data.head()

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
9389,0.207910,0.001334,0.219380,0.058519,0.438924,2.052585e-07,0.167961,0.227325,3.884090e-13,1.0,1.240548e-11,4.355089e-01,0.334721,4.682276e-24,1.0
12010,0.052527,0.770696,0.066655,0.059762,0.616539,2.968356e-01,0.191802,0.603467,7.477485e-18,1.0,2.210325e-13,5.236703e-17,0.472209,1.053886e-08,1.0
16562,0.058534,0.506277,0.092139,0.694673,0.437195,4.725500e-01,0.108754,0.900286,1.241718e-29,0.0,6.910797e-14,4.965952e-05,0.435183,6.820563e-09,1.0
25464,0.105461,0.000012,0.095212,0.286362,0.589724,1.599966e-06,0.227380,0.393798,2.035312e-27,1.0,1.517516e-10,1.472179e-05,0.465225,2.635382e-12,1.0
23379,0.587796,0.661108,0.085556,0.729077,0.441580,1.024449e-03,0.466581,0.494679,1.082536e-25,0.0,2.837474e-10,1.462273e-04,0.668332,9.274970e-08,1.0


In [22]:
eval_model(synth_data, dataset_test)

{'precision': 0.7956058588548602,
 'recall': 0.7956058588548602,
 'auroc': 0.5895699977005224,
 'dp': 0.06265229,
 'ftu': 0.009500027}